In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

from tensorflow.examples.tutorials.mnist import input_data
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#plain

n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

In [4]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')  #act as the input layer
y=tf.placeholder(tf.int32,shape=(None),name='y')

In [5]:
def neuron_layer(X,n_neurons,name,activation=None):
    with tf.name_scope(name):
        n_inputs=int(X.get_shape()[1])
        stddev=2/np.sqrt(n_inputs)
        init=tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
        W=tf.Variable(init,name='weights')
        b=tf.Variable(tf.zeros([n_neurons]),name='biases')
        z=tf.matmul(X,W)+b
        if activation=='relu':
            return tf.nn.relu(z)
        else:
            return z

In [6]:
with tf.name_scope('dnn'):
    hidden1=neuron_layer(X,n_hidden1,'hidden1',activation='relu')
    hidden2=neuron_layer(hidden1,n_hidden2,'hidden2',activation='relu')
    logits=neuron_layer(hidden2,n_outputs,'outputs')

In [7]:
# from tensorflow.contrib.layers import fully_connected

# with tf.name_scope('dnn'):
#     hidden1=fully_connected(X,n_hidden1,scope='hidden1')
#     hidden2=fully_connected(hidden1,n_hidden2,scope='hidden2')
#     output=fully_connected(hidden2,n_outputs,scope='outputs',activation_fn=None)

In [8]:
with tf.name_scope('loss'):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name='loss')

In [9]:
learning_rate=0.01

with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

In [10]:
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [11]:
init = tf.global_variables_initializer()
# saver = tf.train.Saver()

In [12]:
mnist = input_data.read_data_sets("/tmp/data/")

W0702 21:02:16.437479 4731127232 deprecation.py:323] From <ipython-input-12-80aa5e208d67>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0702 21:02:16.438653 4731127232 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0702 21:02:16.442188 4731127232 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future v

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0702 21:02:19.922979 4731127232 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz


W0702 21:02:20.617743 4731127232 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [28]:
n_epochs = 3
batch_size = 50

In [29]:

now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir='tf_logs'
logdir='{}/run-{}'.format(root_logdir,now)
loss_summary=tf.summary.scalar('loss',loss)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

In [30]:

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        if (epoch+1)%4==0:
            summary_str=loss_summary.eval(feed_dict={X:X_batch,y:y_batch})
            file_writer.add_summary(summary_str,epoch)
        print(epoch,'Train accuracy:',acc_train,'Test accuracy:',acc_test)
    file_writer.close()


0 Train accuracy: 0.98 Test accuracy: 0.9136
1 Train accuracy: 0.98 Test accuracy: 0.9266
2 Train accuracy: 0.94 Test accuracy: 0.9351


In [31]:
!ls -l tf_logs/

total 0
drwxr-xr-x  3 trihesdlin  staff  96 Jun 28 16:17 run-20190628201742
drwxr-xr-x  3 trihesdlin  staff  96 Jul  2 21:02 run-20190703010212
drwxr-xr-x  3 trihesdlin  staff  96 Jul  2 21:08 run-20190703010856


In [32]:
!tensorboard --logdir tf_logs/

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.14.0 at http://XiangdongdeMacBook-Pro.local:6006/ (Press CTRL+C to quit)
^C
